In [1]:
# Imprt Dependincies
import pandas as pd
import numpy as np

In [9]:
# Grab CSV Files
schools_file = pd.read_csv("Resources/schools_complete.csv")
schools_df = pd.DataFrame(schools_file)

student_file = pd.read_csv("Resources/students_complete.csv")
student_df = pd.DataFrame(student_file)

schools_df

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [3]:
student_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


### District Summary Table ###

In [81]:
# schools_df.describe

### Total Schools
total_schools = len(schools_df)
#total_schools

### Total Students
student_size = schools_df['size'].sum()
#student_size

### Total Budget
total_budget = schools_df['budget'].sum()
#total_budget

### Average Math Score
avg_math = student_df['math_score'].mean()
avg_math = round(avg_math, 2)
#avg_math

### Average Reading Score
avg_read = student_df['reading_score'].mean()
avg_read = round(avg_read, 2)
#avg_read

### % Passing Math
pass_math = student_df.loc[student_df['math_score'] >= 70]
pass_math = pass_math['math_score'].count()
#pass_math

pass_pct_math = pass_math / student_size * 100
pass_pct_math = round(pass_pct_math, 2)
#pass_pct_math


### % Passing Reading
pass_read = student_df.loc[student_df['reading_score'] >= 70]
pass_read = pass_read['reading_score'].count()
#pass_read

pass_pct_read = pass_read / student_size * 100
pass_pct_read = round(pass_pct_read,2)
#pass_pct_read

student_df['reading_score'].min()

### Overall Passing Rate (Average of the above two)
avg_pass = (pass_pct_math + pass_pct_read) / 2
avg_pass = round(avg_pass, 2)
#avg_pass

### Create a New Summary Table with these values
student_size = '{:,}'.format(student_size)
total_budget = '{0:,.2f}'.format(total_budget)
dist_dict = {"Total Schools " : total_schools,
                          "Total Students " : student_size,
                          "Total Budget " : "$" + total_budget,
                          "Average Math Score " : avg_math,
                          "Average Reading Score " : avg_read,
                          "% Students Passing Math " : str(pass_pct_math) + "%",
                          "% Students Passing Reading " : str(pass_pct_read) + "%",
                          "Overall Passing Rate " : str(avg_pass) + "%"}
index = ["Summary Values"]
dist_df = pd.DataFrame(data=dist_dict, index=index)
dist_df = dist_df.transpose()
dist_df


,Summary Values
Total Schools,15
Total Students,"39,170"
Total Budget,"$24,649,428.00"
Average Math Score,78.99
Average Reading Score,81.88
% Students Passing Math,74.98%
% Students Passing Reading,85.81%
Overall Passing Rate,80.4%


### School Summary Table ###

In [319]:
## School Name
school_name = student_df.groupby(student_df['school_name'])
schoolname = school_name['school_name']
ind_school = schoolname.unique()
#print(ind_school)

## School Type
#school_type = schools_df.groupby(schools_df['school_name'])
school_name = pd.DataFrame(schools_df['school_name'])
school_type = pd.DataFrame(schools_df['type'])
schooltype = school_name.join(school_type)
schooltype = schooltype.sort_values('school_name')
#print(schooltype)

## Total Students
total_students = schools_df['size']
school_size = schooltype.join(total_students)
school_size['size'] = school_size['size'].map('{0:,}'.format)
tot_stu_df = pd.DataFrame(school_size['school_name'])
tot_stu_df = tot_stu_df.join(total_students)

## Total School Budget
total_budget = schools_df['budget']
school_summary = school_size.join(total_budget)
school_summary['budget'] = school_summary['budget'].map('${:,.2f}'.format)

## Per Student Budget
per_student = schools_df['budget'] / schools_df['size']
per_student = per_student.rename('budget per student')
school_summary = school_summary.join(per_student)
school_summary['budget per student'] = school_summary['budget per student'].map('${:,.2f}'.format)
#school_summary

## Average Math Score
math_score = student_df['math_score'].groupby(student_df['school_name'])
avg_math_per = math_score.mean()
avg_math_df = pd.DataFrame(avg_math_per)
school_summary = school_summary.merge(avg_math_df, on="school_name")
school_summary['math_score'] = school_summary['math_score'].map('{:,.2f}%'.format)
#school_summary

## Average Reading Score
read_score = student_df['reading_score'].groupby(student_df['school_name'])
avg_read_per = read_score.mean()
avg_read_df = pd.DataFrame(avg_read_per)
school_summary = school_summary.merge(avg_read_df, on="school_name")
school_summary['reading_score'] = school_summary['reading_score'].map('{:,.2f}%'.format)
#school_summary

## % Passing Math
passing_math = student_df.loc[student_df['math_score'] >= 70]
#print(passing_math)
math_score = passing_math['math_score'].groupby(passing_math['school_name'])
math_score_counts = math_score.count()
math_score_df = pd.DataFrame(math_score_counts)

math_score_per = math_score_df['math_score'] / tot_stu_df['size'].values
math_score_per = math_score_per * 100
math_score_per = round(math_score_per, 2)
math_score_per.rename('% Passing Math')

school_summary = school_summary.merge(math_score_per, on="school_name")
school_summary['math_score_y'] = school_summary['math_score_y'].map('{:,.2f}%'.format)
#school_summary

## % Passing Reading
passing_read = student_df.loc[student_df['reading_score'] >= 70]
read_score = passing_read['reading_score'].groupby(passing_read['school_name'])
read_score_counts = read_score.count()
read_score_df = pd.DataFrame(read_score_counts)

read_score_per = read_score_df['reading_score'] / tot_stu_df['size'].values
read_score_per = read_score_per * 100
read_score_per = round(read_score_per, 2)
read_score_per.rename('% Passing Reading')

school_summary = school_summary.merge(read_score_per, on="school_name")
school_summary['reading_score_y'] = school_summary['reading_score_y'].map('{:,.2f}%'.format)
#school_summary

## Overall Passing Rate (Average of the above two)
avg_pass = (read_score_per + math_score_per) / 2
avg_pass = avg_pass.map('{:,.2f}%'.format)
avg_pass.rename('Average Pass Rate')
avg_pass_df = pd.DataFrame(avg_pass)

school_summary = school_summary.merge(avg_pass_df, on="school_name")
school_summary = school_summary.rename({'school_name': 'School Name', 'type': 'School Type',
                                       'size' : 'School Size', 'budget' : 'Total Budget',
                                       'math_score_x' : 'Average Math Score', 'reading_score_x' : "Average Reading Score",
                                       'math_score_y' : '% Passing Math', 'reading_score_y' : "% Passing Reading",
                                       0 : 'Overall Pass Rate'}, axis='columns')
school_summary

,School Name,School Type,School Size,Total Budget,budget per student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05%,81.03%,66.68%,81.93%,74.31%
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.81%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29%,80.93%,66.75%,80.86%,73.81%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.80%,83.81%,92.51%,96.25%,94.38%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%


### Top Performing Schools ###

In [321]:
top_schools = school_summary.sort_values(by='Overall Pass Rate', ascending=False).head(5)
top_schools

,School Name,School Type,School Size,Total Budget,budget per student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
12,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,95.29%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%
13,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,95.21%


### Worst Performing Schools ###

In [322]:
worst_schools = school_summary.sort_values(by='Overall Pass Rate', ascending=True).head(5)
worst_schools

,School Name,School Type,School Size,Total Budget,budget per student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
10,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,73.30%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.81%


### Math Scores by Grade ###

In [345]:
## pull students by grade level
ninth_grade = student_df.loc[student_df['grade'] == "9th"]
tenth_grade = student_df.loc[student_df['grade'] == "10th"]
eleven_grade = student_df.loc[student_df['grade'] == "11th"]
twelve_grade = student_df.loc[student_df['grade'] == "12th"]

nine_group = ninth_grade.groupby(student_df['school_name'])
nine_math = nine_group['math_score'].mean()
nine_math = nine_math.map('{:,.2f}%'.format)

ten_group = tenth_grade.groupby(student_df['school_name'])
ten_math = ten_group['math_score'].mean()
ten_math = ten_math.map('{:,.2f}%'.format)

eleven_group = eleven_grade.groupby(student_df['school_name'])
eleven_math = eleven_group['math_score'].mean()
eleven_math = eleven_math.map('{:,.2f}%'.format)

twelve_group = twelve_grade.groupby(student_df['school_name'])
twelve_math = twelve_group['math_score'].mean()
twelve_math = twelve_math.map('{:,.2f}%'.format)

math_by_grade = pd.DataFrame(nine_math)
math_by_grade = math_by_grade.rename({'math_score' : '9th'}, axis = 'columns')
math_by_grade = math_by_grade.merge(ten_math, on="school_name")
math_by_grade = math_by_grade.rename({'math_score' : '10th'}, axis = 'columns')
math_by_grade = math_by_grade.merge(eleven_math, on="school_name")
math_by_grade = math_by_grade.rename({'math_score' : '11th'}, axis = 'columns')
math_by_grade = math_by_grade.merge(twelve_math, on="school_name")
math_by_grade = math_by_grade.rename({'math_score' : '12th'}, axis = 'columns')
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


### Reading Scores by Grade ###

In [346]:
# copy/paste from above
nine_read = nine_group['reading_score'].mean()
nine_read = nine_read.map('{:,.2f}%'.format)

ten_read = ten_group['reading_score'].mean()
ten_read = ten_read.map('{:,.2f}%'.format)

eleven_read = eleven_group['reading_score'].mean()
eleven_read = eleven_read.map('{:,.2f}%'.format)

twelve_read = twelve_group['reading_score'].mean()
twelve_read = twelve_read.map('{:,.2f}%'.format)

read_by_grade = pd.DataFrame(nine_read)
read_by_grade = read_by_grade.rename({'reading_score' : '9th'}, axis = 'columns')
read_by_grade = read_by_grade.merge(ten_read, on="school_name")
read_by_grade = read_by_grade.rename({'reading_score' : '10th'}, axis = 'columns')
read_by_grade = read_by_grade.merge(eleven_read, on="school_name")
read_by_grade = read_by_grade.rename({'reading_score' : '11th'}, axis = 'columns')
read_by_grade = read_by_grade.merge(twelve_read, on="school_name")
read_by_grade = read_by_grade.rename({'reading_score' : '12th'}, axis = 'columns')
read_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


### Scores by School Spending ###

In [422]:
#Average Math Score
#Average Reading Score
#% Passing Math
#% Passing Reading
#Overall Passing Rate (Average of the above two)

# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

spend_df = pd.cut(per_student, bins = spending_bins, labels = group_names)
new_summary_df = school_summary.join(spend_df, rsuffix = "_new")
new_summary_df['Average Math Score'] = new_summary_df['Average Math Score'].str.rstrip('%').astype('float')
new_summary_df['Average Reading Score'] = new_summary_df['Average Reading Score'].str.rstrip('%').astype('float')
new_summary_df['% Passing Math'] = new_summary_df['% Passing Math'].str.rstrip('%').astype('float')
new_summary_df['% Passing Reading'] = new_summary_df['% Passing Reading'].str.rstrip('%').astype('float')
new_summary_df['Overall Pass Rate'] = new_summary_df['Overall Pass Rate'].str.rstrip('%').astype('float')
new_summary = new_summary_df.groupby('budget per student_new')
group_spend_df = new_summary.mean()
group_spend_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
budget per student_new,,,,,
<$585,78.75,81.6125,72.9225,84.63750000000002,78.7825
$585-615,80.275,82.6,80.28999999999999,88.345,84.315
$615-645,82.225,83.44166666666668,89.045,94.08333333333333,91.56666666666666
$645-675,79.19,81.87666666666667,76.08666666666666,86.18,81.13666666666667


### Scores by School Size ###

In [421]:
# Same as above, just based on school size
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

size_df = pd.cut(total_students, bins = size_bins, labels = group_names)
new_summ_df = school_summary.join(size_df)
new_summ_df['Average Math Score'] = new_summ_df['Average Math Score'].str.rstrip('%').astype('float')
new_summ_df['Average Reading Score'] = new_summ_df['Average Reading Score'].str.rstrip('%').astype('float')
new_summ_df['% Passing Math'] = new_summ_df['% Passing Math'].str.rstrip('%').astype('float')
new_summ_df['% Passing Reading'] = new_summ_df['% Passing Reading'].str.rstrip('%').astype('float')
new_summ_df['Overall Pass Rate'] = new_summ_df['Overall Pass Rate'].str.rstrip('%').astype('float')
new_summ = new_summ_df.groupby('size')
group_size_df = new_summ.mean()
group_size_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
size,,,,,
Small (<1000),80.455,82.505,80.325,88.58500000000001,84.455
Medium (1000-2000),80.876,82.696,82.318,90.19200000000001,86.256
Large (2000-5000),80.1475,82.43,80.32000000000001,88.76875000000001,84.5475


### Scores by School Type ###

In [439]:
# Same as above, baes on school type: District or Charter

dist_df = school_summary.loc[school_summary['School Type'] == "District"]
char_df = school_summary.loc[school_summary['School Type'] == "Charter"]

dist_type_df = dist_df.copy()
dist_type_df['Average Math Score'] = dist_type_df['Average Math Score'].str.rstrip('%').astype('float')
dist_type_df['Average Reading Score'] = dist_type_df['Average Reading Score'].str.rstrip('%').astype('float')
dist_type_df['% Passing Math'] = dist_type_df['% Passing Math'].str.rstrip('%').astype('float')
dist_type_df['% Passing Reading'] = dist_type_df['% Passing Reading'].str.rstrip('%').astype('float')
dist_type_df['Overall Pass Rate'] = dist_type_df['Overall Pass Rate'].str.rstrip('%').astype('float')
group_dist_df = dist_type_df.mean()
group_dist_df = group_dist_df.rename("District")
group_dist_df

char_type_df = char_df.copy()
char_type_df['Average Math Score'] = char_type_df['Average Math Score'].str.rstrip('%').astype('float')
char_type_df['Average Reading Score'] = char_type_df['Average Reading Score'].str.rstrip('%').astype('float')
char_type_df['% Passing Math'] = char_type_df['% Passing Math'].str.rstrip('%').astype('float')
char_type_df['% Passing Reading'] = char_type_df['% Passing Reading'].str.rstrip('%').astype('float')
char_type_df['Overall Pass Rate'] = char_type_df['Overall Pass Rate'].str.rstrip('%').astype('float')
group_char_df = char_type_df.mean()
group_char_df = group_char_df.rename("Charter")
group_char_df

type_df = pd.DataFrame(group_dist_df)
type_df = type_df.join(group_char_df)
type_df = type_df.transpose()
type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
District,76.95571428571428,80.96571428571428,66.54857142857144,80.79857142857144,73.67571428571429
Charter,83.4725,83.89625000000001,93.62,96.58624999999999,95.105


#### You must include a written description of at least two observable trends based on the data. ####

* Based on the rankings of Top and Bottom performing schools, the data seems to should that Charter schools tend to outperform District schools. This is most easily shown in the direct analysis between the two, where Charter schools outperform District schools in every caregory measured. However, further analysis is needed in order ot determine if other outlying factors weigh into the phenomenon (parental support, social status, etc.).

* Based on all budget data, there seems to be a normal distribution in effectiveness of budget spent per student. In effect, the data reflects that student performance tapers off when too little or too much money is budgeted per student. This is best shown in the budget ranking analysis. However, further analysis is needed in order ot determine if other outlying factors weigh into the phenomenon (environmental conditions, other indv. school expenses, etc.).